In [ ]:
# ============================================
# Chapter 1: Time Series Data
# ============================================

# 문제 0. 라이브러리 호출
import pandas as pd  # 데이터 처리 및 분석
import numpy as np   # 수치 계산 및 배열 연산
import seaborn as sns  # 통계적 데이터 시각화
import matplotlib.pyplot as plt  # 기본 시각화


# 문제 1. 데이터 파일 불러오기 & Date 열 datetime 형식 설정
df = pd.read_csv("bike_sharing.csv")  # CSV 파일 불러오기
df['Date'] = pd.to_datetime(df['Date'])  # 'Date'를 datetime 형식으로 변환
df.set_index('Date', inplace=True)  # 'Date'를 인덱스로 설정
df.head(5)  # 상위 5개 행 확인


# 문제 2. 데이터 구조 확인
df.info()      # 데이터프레임 정보 확인
df.describe()  # 수치형 변수 요약 통계 확인


# 문제 3. 자전거 대여량의 분포 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Seasons별 대여량
sns.boxplot(x='Seasons', y='Rented Bike Count', data=df, ax=axes[0])
axes[0].set_title('Rented Bike Count by Seasons')
# 해석:
# Winter: 대여량 낮음, 일부 이상치
# Spring, Autumn: 평균/중간값 증가, Autumn 분포 넓음
# Summer: 대여량 최고, 이용 활발

# Holiday 여부별 대여량
sns.boxplot(x='Holiday', y='Rented Bike Count', data=df, ax=axes[1])
axes[1].set_title('Rented Bike Count by Holiday')
# 해석:
# No Holiday: 분포 넓고 평균/중간값 높음
# Holiday: 대여량 낮음, 이상치 적음

# Functioning Day 여부별 대여량
sns.boxplot(x='Functioning Day', y='Rented Bike Count', data=df, ax=axes[2])
axes[2].set_title('Rented Bike Count by Functioning Day')
# 해석:
# Yes: 다양하고 정상 분포
# No: 대여량 거의 0, 시스템 비활성화 또는 데이터 미수집

plt.tight_layout()
plt.show()


# 문제 4. 운영일이면서 대여량이 0인 날 확인
df_zero_rent = df[(df['Functioning Day'] == 'Yes') & (df['Rented Bike Count'] == 0)]
df_zero_rent
# 해석: 운영일이면서 대여량이 0인 날은 없음


# 문제 5. 운영일이 아닌 날 대여량이 0인 연속 기간 확인
zero_rent_df = df[df['Rented Bike Count'] == 0].copy()
zero_rent_df['gap'] = zero_rent_df.index.to_series().diff().gt('1H').cumsum()
zero_rent_periods = zero_rent_df.groupby('gap').agg(
    Date=('Rented Bike Count', lambda x: x.index.min()),
    Count=('Rented Bike Count', 'count')
).reset_index(drop=True)
display(zero_rent_periods)
print(f"총 대여량이 0이었던 연속 기간 수: {len(zero_rent_periods)}")
# 해석: 운영되지 않은 날들의 연속 기간을 확인 가능


# 문제 6. 대여량 0 제거
num_zero = sum(df['Rented Bike Count'] == 0)
num_zero  # 0인 행 수 확인

ratio_zero = num_zero / len(df)
ratio_zero  # 전체에서 차지하는 비율 확인

# BONUS 해석:
# - 대여량 0인 행은 전체의 약 3.4% 정도로 매우 적음
# - 운영일이 아닌 경우에만 해당하며 특별한 계절/공휴일 특성 없음
# - 단순히 운영하지 않은 날로 판단하고 제거해도 됨
df = df[df['Rented Bike Count'] != 0]  # 0인 행 제거

# 제거 후 확인 (대여량 0인 행이 없는지)
df[df['Rented Bike Count'] == 0]


# 문제 7. 상관계수 히트맵
numeric_df = df.select_dtypes(include=['float64', 'int64'])  # 수치형 변수만 선택
corr_matrix = numeric_df.corr()  # 상관계수 계산

plt.figure(figsize=(12, 12))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Pearson Correlation Heatmap")
plt.tight_layout()
plt.show()
# 해석:
# - 상관계수 히트맵을 통해 수치형 변수 간 선형 관계 확인
# - Rented Bike Count와 기상 변수(Temperature, Humidity 등) 관계 확인에 유용


# 문제 8. 변수 제거
# Dew point temperature(°C)와 Temperature(°C)가 매우 높은 상관관계를 가짐
# Temperature를 유지하고 Dew point를 제거하여 다중공선성 방지
df = df.drop(columns=['Dew point temperature(°C)'])
# 해석:
# - Temperature(°C)와 강한 상관을 가지므로 Dew point temperature를 제거
# - 모델 학습 시 중복 정보를 줄이고 해석 용이성 향상


In [ ]:
# ============================================
# Chapter 2: Multivariate LSTM
# ============================================

# 문제 0. 라이브러리 불러오기
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 문제 1. 대여량을 y에 저장
y = df['Rented Bike Count'].values  # 타깃 변수 설정 (예측할 값)


# 문제 2. 범주형 변수에 원-핫 인코딩
categorical_cols = ['Seasons', 'Holiday', 'Functioning Day']  # 범주형 변수 선택
encoder = OneHotEncoder(sparse_output=False)  # 원핫 인코더 초기화
X_cat = encoder.fit_transform(df[categorical_cols])  # 원핫 인코딩 수행

# 나머지 수치형 변수 추출
numerical_cols = df.drop(columns=categorical_cols + ['Rented Bike Count']).columns
X_num = df[numerical_cols].values

# 수치형 + 인코딩된 범주형 합치기
X = np.concatenate([X_num, X_cat], axis=1)

# BONUS: 선형모델 계열에서는 drop_first=True 필요하지만,
# 딥러닝 모델에서는 그대로 사용 가능


# 문제 3. Train/Test 데이터 셋 split
split_index = int(len(X) * 0.8)  # 8:2 split, 시계열 순서 유지
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
X.shape


# 문제 4. 정규화
from sklearn.preprocessing import StandardScaler

# y가 1차원이므로 reshape
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

# train 데이터로만 fit
scaler_X.fit(X_train)
scaler_y.fit(y_train)

# transform
X_train_scaled = scaler_X.transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


# 문제 5. Windowing 작업
# 윈도잉(Windowing) 작업은 LSTM 모델에 입력할 시계열 데이터 시퀀스를 생성하기 위해 필요함
# LSTM은 과거 'window_size' 만큼의 시퀀스를 보고 다음 시점 값을 예측하기 때문
def create_sequences(X, y, window_size=24):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size):
        X_seq.append(X[i:i+window_size])       # 과거 window_size 시점의 입력
        y_seq.append(y[i+window_size])         # window 이후 시점의 타깃
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, window_size=24)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, window_size=24)


# 문제 6. train_loader 객체 생성
# DataLoader는 배치 단위 학습, 셔플, 반복 등 편리한 기능 제공
# LSTM 학습 시 시계열 순서 유지 위해 shuffle=False
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

train_dataset = SequenceDataset(X_train_seq, y_train_seq)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
# train_loader 객체를 통해 배치 학습 가능, 메모리 효율적 처리 및 반복 학습 지원


# 문제 7. LSTM model 객체 생성
# LSTM 모델은 시계열 데이터를 다루며 과거 시점 정보를 기억하여 다음 시점 예측 가능
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.act = nn.LeakyReLU(0.1)  # 비선형 활성화 함수

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 마지막 시점 hidden state를 사용
        return self.act(out).squeeze()

input_dim = X_train_seq.shape[2]
model = LSTMModel(input_dim=input_dim, hidden_dim=64, num_layers=2)


# 문제 8. LSTM 모델 훈련
X_train_seq = torch.tensor(X_train_seq, dtype=torch.float32)
y_train_seq = torch.tensor(y_train_seq, dtype=torch.float32)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(15):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")


# 문제 9. Test set 예측
model.eval()
X_test_seq_tensor = torch.tensor(X_test_seq, dtype=torch.float32)

with torch.no_grad():
    y_pred_scaled = model(X_test_seq_tensor)


# 문제 10. 예측값과 실제값 역정규화
# 역정규화(Inverse Scaling)는 모델이 학습 시 정규화된 값으로 예측하므로,
# 실제 단위(자전거 대여량)로 변환하여 해석과 시각화를 가능하게 함
y_test_inv = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1))
y_pred_inv = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))


# 문제 11. 시각화
import pandas as pd
import matplotlib.pyplot as plt

window_size = 24
split_index = int(len(df) * 0.8)

test_df = df.iloc[split_index + window_size:].copy()
test_df['Datetime'] = pd.to_datetime(test_df.index) + pd.to_timedelta(test_df['Hour'], unit='h')

time_series = test_df['Datetime'].iloc[:len(y_test_inv)].reset_index(drop=True)

y_test_flat = y_test_inv.flatten()
y_pred_flat = y_pred_inv.flatten()

plt.figure(figsize=(14, 5))
plt.plot(time_series, y_test_flat, label='Actual', color='blue')
plt.plot(time_series, y_pred_flat, label='Predicted', color='red')
plt.xlabel('Time')
plt.ylabel('Rented Bike Count')
plt.title('LSTM Forecast vs Actual (Inverse Scaled)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
